<a href="https://colab.research.google.com/github/Alpha-lab12/Alpha-careers-website/blob/main/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests # This library helps to fetch data from API
import pandas as pd # Helps for handling and analysing data
import numpy as np # Used to perform numerical operation
from sklearn.model_selection import train_test_split # to split data into training and testing
from sklearn.preprocessing import LabelEncoder # to convert categorical data to numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor #model for classification and regression
from sklearn.metrics import mean_squared_error #For measuring accuracy of the model/prediction
from datetime import datetime, timedelta #to handle date and time
import pytz


In [4]:
API_KEY = '08b881ad5e8964d704fa0f9cfc06dec4' #Replace with the actual API keys
BASE_URL = 'https://api.openweathermap.org/data/2.5/' #Base URL for making API requests


1. Function to fetch the current weather data

In [16]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"

    response = requests.get(url)
    data = response.json()

    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_speed': round(data['wind']['speed']),
        'cloud_cover': round(data['clouds']['all']),
        'pressure': round(data['main']['pressure'])
    }
    return data

2. Read Historical Data

In [15]:
def read_historical_data(filename):
  df = pd.read_csv(filename) #load csv file into Dataframe
  df = df.dropna()
  df = df.drop_duplicates() # Added parentheses here
  df.head()
  return df

3. Prepare data for training

In [7]:
from sklearn.preprocessing import LabelEncoder

def prepare_data(data):
    # Initialize and apply the label encoder
    le = LabelEncoder()
    data['Rain'] = le.fit_transform(data['Rain'])

    # Define features and target
    x = data[['Temperature', 'Humidity', 'Wind_Speed', 'Cloud_Cover', 'Pressure']]
    y = data['Rain']

    return x, y, le



4: Train Rain model prediction

In [17]:
def train_rain_model(x,y):
 x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
 model=RandomForestClassifier(n_estimators = 100, random_state =42)
 model.fit(x_train, y_train)
 y_pred = model.predict(x_test)
 print("Mean Squared error for rain model ")
 print(mean_squared_error(y_test, y_pred))
 return model


**5: Prepare regression model**

In [18]:
def prepare_regression_data(data, feature):
    x, y = [], []

    # Build pairs: current value vs. next value
    for i in range(len(data) - 1):
        x.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])

    # Convert to NumPy arrays
    x = np.array(x).reshape(-1, 1)  # Feature needs to be 2D for most regressors
    y = np.array(y)  # Target values (next time step)

    return x, y

**6: Train regression model**

In [10]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x,y)
  return model

**7: Predict future**

In [11]:
def predict_future(model, current_value):
  predictions =[current_value]
  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])
  return predictions[1:] # Moved return statement outside the loop

**8: Weather analysis function**

In [12]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
# 🌍 Main weather view function
def weather_view():
    city = input("Enter any city name: ")

    current_weather = get_current_weather(city)
    historical_data = read_historical_data("/content/weather_forecast_data.csv")

    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    current_data = {
        "Temperature": current_weather["current_temp"], # Changed 'Temp' to 'Temperature'
        "Humidity": current_weather["humidity"],
        "Wind_Speed": current_weather["wind_speed"],
        "Cloud_Cover": current_weather["cloud_cover"],
        "Pressure": current_weather["pressure"]
    }

    current_df = pd.DataFrame([current_data])

#Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

#Prepare regression model for temperature and humidity

    x_temp, y_temp = prepare_regression_data(historical_data, 'Temperature') # Changed 'Temp' to 'Temperature'
    x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(x_temp, y_temp)
    hum_model = train_regression_model(x_hum, y_hum)

#Predict future temperature and humidity

    future_temp = predict_future(temp_model, current_weather['current_temp'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

#prepare time for future prediction

    timezone = pytz.timezone('Africa/Dar_es_Salaam')
    now = datetime.now(timezone)
    next_hours = now.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    future_times = [(next_hours + timedelta(hours=i)).strftime('%H:00') for i in range(1, 6)]

    # 🖥️ Display results
    print("\n=== Weather Advisory ===")
    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Wind Speed: {current_weather['wind_speed']} km/h")
    print(f"Cloud Cover: {current_weather['cloud_cover']}%")
    print(f"Pressure: {current_weather['pressure']} hPa")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Rain expected 🌧️' if rain_prediction else 'No rain ☀️'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, hum in zip(future_times, future_humidity):
        print(f"{time}: {round(hum, 1)}%")

# Call function
weather_view() # Commenting out the direct call to weather_view() within the function definition to avoid automatic execution upon function definition.

Enter any city name: mbeya
Mean Squared error for rain model 
0.002

=== Weather Advisory ===
City: mbeya, TZ
Current Temperature: 16°C
Feels Like: 16
Humidity: 57%
Wind Speed: 2 km/h
Cloud Cover: 31%
Pressure: 1021 hPa
Weather Prediction: scattered clouds
Rain Prediction: No rain ☀️

Future Temperature Predictions:
11:00: 17.9°C
12:00: 30.9°C
13:00: 16.1°C
14:00: 17.6°C
15:00: 23.1°C

Future Humidity Predictions:
11:00: 50.0%
12:00: 59.9%
13:00: 46.9%
14:00: 47.8%
15:00: 82.8%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
weather_view()

Enter any city name: dodoma
Mean Squared error for rain model 
0.002

=== Weather Advisory ===
City: dodoma, TZ
Current Temperature: 20°C
Feels Like: 19
Humidity: 48%
Wind Speed: 3 km/h
Cloud Cover: 54%
Pressure: 1020 hPa
Weather Prediction: broken clouds
Rain Prediction: No rain ☀️

Future Temperature Predictions:
11:00: 24.6°C
12:00: 25.7°C
13:00: 25.9°C
14:00: 29.2°C
15:00: 24.9°C

Future Humidity Predictions:
11:00: 63.9%
12:00: 57.5%
13:00: 69.5%
14:00: 57.1%
15:00: 49.5%


In [14]:
weather_view()

Enter any city name: iringa
Mean Squared error for rain model 
0.002

=== Weather Advisory ===
City: iringa, TZ
Current Temperature: 18°C
Feels Like: 17
Humidity: 50%
Wind Speed: 2 km/h
Cloud Cover: 69%
Pressure: 1021 hPa
Weather Prediction: broken clouds
Rain Prediction: No rain ☀️

Future Temperature Predictions:
11:00: 19.3°C
12:00: 19.9°C
13:00: 19.0°C
14:00: 20.7°C
15:00: 16.6°C

Future Humidity Predictions:
11:00: 75.3%
12:00: 68.5%
13:00: 60.2%
14:00: 83.3%
15:00: 44.7%
